<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-06-06 10:23:51
-------------------
qualified stocks: 93
with latest results: 31
still star stocks: 17
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  1.64 L
Current:  1.33 C
-------------------
Today PnL: 39.91 K (0.3%)
Current PnL: -16.09 L (-11.21%)
CY Booked + Current PnL: -11.09 L (-7.73%)
-------------------
Total profit:  4.32 L
Total loss:  -20.41 L
-------------------
Total Booked + Current PnL: 15.89 L (13.63%)
Total Booked PnL: 31.97 L (27.43%)
Curr Year Booked PnL: 4.99 L (3.77%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.12 C
Est FTT PnL: 78.97 L (59.58%)
-------------------
Est LTT:  2.74 C
Est LTT PnL: 1.41 C (106.52%)
Deployed:  1.17 C
Current:  1.33 C
CAGR/XIRR %: 9.14%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LICI,785.05,983.0,-3.65,H-LC,89.25,236927.0,42235.0,6847.0,-0.13,21.69,2.89,25.21,31.0,6.17,1.81,31.01,XY25,NTT,INSURANCE
4,ADANIPORTS,1103.69,1583.0,3.60,M-LC,64.52,104022.0,25660.0,8374.0,0.58,32.75,8.05,43.43,70.0,3.06,0.79,39.13,XY24,NTT,MISC
62,RELIANCE,1291.83,1526.0,6.12,X-LC,26.88,157712.0,16903.0,8627.0,0.31,12.00,5.47,18.13,19.0,1.96,1.20,24.53,XY25,NTT,REFINERIES
56,PFIZER,4245.50,6318.3,14.35,H-SC,97.85,132078.0,34432.0,13247.0,-0.94,35.26,10.03,48.82,134.0,2.60,1.01,46.73,X40,ATH,PHARMA
85,UNIONBANK,123.87,163.0,-1.84,M-LC,93.55,171096.0,30256.0,14235.0,-1.36,21.48,8.32,31.59,79.0,2.13,1.31,48.62,XY24,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ASIANTILES,75.41,137.00,6755.56,M-SC,98.92,75200.0,-18610.0,95226.0,5.35,-19.84,126.63,81.67,214.0,-0.20,0.57,45.63,XR,NTT,CERAMICS
64,ROUTE,1558.40,2249.26,-38.28,M-SC,41.94,66469.0,-36385.0,81983.0,4.49,-35.38,123.34,44.33,172.0,-0.44,0.51,10.38,SR,ATH,IT
26,DLF,714.31,885.00,-10.40,H-LC,92.47,273307.0,36870.0,19623.0,2.78,15.59,7.18,23.90,30.0,1.88,2.09,35.79,AR,NTT,REALTY
34,HEROMOTOCO,4311.81,6039.03,-1.90,H-MC,59.14,149464.0,-1449.0,61908.0,2.19,-0.96,41.42,40.06,86.0,-0.02,1.14,22.25,AR,ATH,AUTO
63,REPCOHOME,515.07,880.00,-56.04,H-SC,53.76,162145.0,-42853.0,188088.0,2.18,-20.90,116.00,70.85,121.0,-0.23,1.24,30.89,XY24,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,SONACOMS,578.05,1006.0,-30.24,M-MC,33.33,90212.0,-10947.0,85837.0,-2.39,-10.82,95.15,74.03,155.0,-0.13,0.69,26.78,AR,BTT,AUTO
87,VALIANTORG,512.64,838.0,-367.00,H-SC,100.00,113425.0,-19861.0,104453.0,-2.03,-14.90,92.09,63.47,130.0,-0.19,0.87,90.13,XR,NTT,CHEMICALS
72,SIS,477.00,528.0,2229.56,M-SC,77.96,63062.0,-18028.0,26700.0,-1.49,-22.23,42.34,10.69,201.0,-0.68,0.48,27.80,X40N,NTT,MISC
85,UNIONBANK,123.87,163.0,-1.84,M-LC,93.55,171096.0,30256.0,14235.0,-1.36,21.48,8.32,31.59,79.0,2.13,1.31,48.62,XY24,NTT,BANKS
52,LUXIND,1448.19,2442.0,37.89,M-SC,68.82,143222.0,5644.0,88769.0,-1.28,4.10,61.98,68.62,183.0,0.06,1.09,24.16,X40N,NTT,TEXTILES


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,1461.46,2275.00,-13.50,X-LC,35.48,277271.0,15670.0,129957.0,-0.34,5.99,46.87,55.67,1.0,0.12,2.12,12.46,X40,BTT,IT
80,TCS,3794.03,4998.00,-19.05,X-LC,15.05,263406.0,-32528.0,126435.0,0.18,-10.99,48.00,31.73,2.0,-0.26,2.01,5.43,X40,BTT,IT
51,LTIM,5564.16,7197.33,-3.08,X-LC,77.96,186768.0,-13542.0,72335.0,0.13,-6.76,38.73,29.35,3.0,-0.19,1.43,29.03,X200,ATH,IT
7,ASIANPAINT,2961.56,4250.00,-23.13,X-LC,4.30,152884.0,-48502.0,136113.0,0.21,-24.08,89.03,43.51,5.0,-0.36,1.17,5.45,X40,BTT,PAINTS
35,HINDUNILVR,2413.81,2723.00,-14.85,X-LC,18.28,267467.0,-2880.0,37499.0,0.49,-1.07,14.02,12.81,6.0,-0.08,2.04,10.70,XY25,NTT,FMCG


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,PFIZER,4245.50,6318.30,14.35,H-SC,97.85,132078.0,34432.0,13247.0,-0.94,35.26,10.03,48.82,134.0,2.60,1.01,46.73,X40,ATH,PHARMA
57,POLYCAB,5005.95,7541.35,7.93,H-MC,27.96,102382.0,17281.0,25821.0,0.70,20.31,25.22,50.65,90.0,0.67,0.78,28.70,X40N,ATH,CABLES
85,UNIONBANK,123.87,163.00,-1.84,M-LC,93.55,171096.0,30256.0,14235.0,-1.36,21.48,8.32,31.59,79.0,2.13,1.31,48.62,XY24,NTT,BANKS
4,ADANIPORTS,1103.69,1583.00,3.60,M-LC,64.52,104022.0,25660.0,8374.0,0.58,32.75,8.05,43.43,70.0,3.06,0.79,39.13,XY24,NTT,MISC
50,LICI,785.05,983.00,-3.65,H-LC,89.25,236927.0,42235.0,6847.0,-0.13,21.69,2.89,25.21,31.0,6.17,1.81,31.01,XY25,NTT,INSURANCE


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SAIL,130.64,228.00,100.99,M-MC,87.10,228905.0,3943.0,163713.0,-0.14,1.75,71.52,74.53,170.0,0.02,1.75,32.90,XY24,BTT,STEEL
52,LUXIND,1448.19,2442.00,37.89,M-SC,68.82,143222.0,5644.0,88769.0,-1.28,4.10,61.98,68.62,183.0,0.06,1.09,24.16,X40N,NTT,TEXTILES
56,PFIZER,4245.50,6318.30,14.35,H-SC,97.85,132078.0,34432.0,13247.0,-0.94,35.26,10.03,48.82,134.0,2.60,1.01,46.73,X40,ATH,PHARMA
57,POLYCAB,5005.95,7541.35,7.93,H-MC,27.96,102382.0,17281.0,25821.0,0.70,20.31,25.22,50.65,90.0,0.67,0.78,28.70,X40N,ATH,CABLES
18,CAMS,3643.00,5250.99,6.22,H-SC,86.02,119364.0,17360.0,27669.0,0.35,17.02,23.18,44.14,112.0,0.63,0.91,36.02,XR,ATH,MISC


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,3643.00,5250.99,6.22,H-SC,86.02,119364.0,17360.0,27669.0,0.35,17.02,23.18,44.14,112.0,0.63,0.91,36.02,XR,ATH,MISC
12,BANKINDIA,113.49,190.00,-28.22,M-MC,69.89,140384.0,11232.0,75835.0,-0.72,8.70,54.02,67.42,156.0,0.15,1.07,36.54,XR,NTT,BANKS
39,INDIAMART,2327.09,4850.92,-51.09,H-SC,55.91,127189.0,3853.0,129911.0,-0.33,3.12,102.14,108.45,108.0,0.03,0.97,26.18,AR,ATH,MISC
92,WIPRO,243.46,420.00,-10.16,M-LC,34.41,154033.0,3088.0,106360.0,0.20,2.05,69.05,72.51,68.0,0.03,1.18,9.92,XR,NTT,IT
1,AARTIIND,487.04,919.00,39.67,M-SC,60.22,80193.0,-1143.0,73280.0,-0.30,-1.40,91.38,88.69,205.0,-0.02,0.61,35.23,XR,NTT,CHEMICALS


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MASFIN,326.60,399.5,-23.67,H-SC,56.99,86580.0,-11400.0,33273.0,1.37,-11.64,38.43,22.32,127.0,-0.34,0.66,25.57,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.0,3.60,M-LC,64.52,104022.0,25660.0,8374.0,0.58,32.75,8.05,43.43,70.0,3.06,0.79,39.13,XY24,NTT,MISC
87,VALIANTORG,512.64,838.0,-367.00,H-SC,100.00,113425.0,-19861.0,104453.0,-2.03,-14.90,92.09,63.47,130.0,-0.19,0.87,90.13,XR,NTT,CHEMICALS
84,UJJIVANSFB,52.77,60.0,41.19,M-SC,62.37,122067.0,-20412.0,39928.0,1.60,-14.33,32.71,13.70,221.0,-0.51,0.93,44.07,X40N,NTT,BANKS
56,PFIZER,4245.50,6318.3,14.35,H-SC,97.85,132078.0,34432.0,13247.0,-0.94,35.26,10.03,48.82,134.0,2.60,1.01,46.73,X40,ATH,PHARMA


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VALIANTORG,512.64,838.0,-367.00,H-SC,100.00,113425.0,-19861.0,104453.0,-2.03,-14.90,92.09,63.47,130.0,-0.19,0.87,90.13,XR,NTT,CHEMICALS
8,ASIANTILES,75.41,137.0,6755.56,M-SC,98.92,75200.0,-18610.0,95226.0,5.35,-19.84,126.63,81.67,214.0,-0.20,0.57,45.63,XR,NTT,CERAMICS
56,PFIZER,4245.50,6318.3,14.35,H-SC,97.85,132078.0,34432.0,13247.0,-0.94,35.26,10.03,48.82,134.0,2.60,1.01,46.73,X40,ATH,PHARMA
81,TITAGARH,1097.23,1548.0,-7.11,H-SC,96.77,163188.0,-28827.0,107720.0,-0.15,-15.01,66.01,41.08,128.0,-0.27,1.25,38.12,XY24,NTT,ENGINEERING
74,STARHEALTH,551.56,761.0,4.42,M-SC,95.70,170556.0,-29660.0,105694.0,-0.14,-14.81,61.97,37.97,195.0,-0.28,1.30,36.94,XY24,NTT,INSURANCE


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.74
1,25,42.08
2,50,70.53


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    31.58
AR      15.08
XY25    14.63
X40N    13.27
X40     12.48
XR      10.36
X200     1.43
SR       1.15
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.01
X-LC    18.05
M-SC    17.42
H-LC     9.22
H-MC     8.55
M-LC     7.69
X-MC     5.53
M-MC     5.50
X-SC     2.76
L-SC     2.29
L-LC     1.13
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.46,-1.60,33.41
IT,10.22,-16.78,80.21
BANKS,9.60,-11.39,62.59
MISC,6.50,-26.89,82.64
FINANCE,5.85,-24.53,83.67
PAINTS,5.30,-18.75,53.90
INSURANCE,5.29,3.72,25.05
ELECTRICAL,5.15,-4.29,49.53
HEALTHCARE,4.35,-4.29,35.41


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2841814.0,26
AR,1247957.0,14
X40N,1209279.0,17
XR,1095188.0,14
X40,774793.0,9
XY25,461117.0,10
SR,194511.0,2
X200,72335.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2219021.0,20
M-SC,2032723.0,21
X-LC,848170.0,11
M-MC,548359.0,5
H-MC,508213.0,8
X-SC,347327.0,4
X-MC,309149.0,5
M-LC,298257.0,6
L-SC,298230.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      925956.0      6
M-SC       XY24      759465.0      6
H-SC       AR        539220.0      3
           X40N      499431.0      6
X-LC       X40       481998.0      4
M-SC       X40N      471375.0      6
M-MC       XY24      386687.0      3
M-SC       XR        286564.0      4
H-SC       XR        241167.0      4
H-LC       AR        238012.0      5
M-SC       X40       218599.0      2
M-LC       XY24      177195.0      4
H-MC       XY24      173000.0      2
X-LC       AR        167408.0      2
L-SC       XR        157831.0      2
X-MC       XY24      139853.0      2
X-LC       XY25      126429.0      4
M-SC       XY25      120507.0      1
X-SC       X40N      119895.0      2
           XR        114904.0      1
           SR        112528.0      1
H-MC       XR        112527.0      1
M-LC       XR        106360.0      1
M-SC       AR         94230.0      1
X-MC       X40N       92757.0      2
M-MC       AR         85837.0      1
M-SC       SR         81983.0      1
L-SC       XY24       79057.0      1
X-MC       XY25       76539.0      1
M-MC       XR         75835.0      1
H-MC       XY25       74008.0      1
X-LC       X200       72335.0      1
H-MC       AR         61908.0      1
L-SC       AR         61342.0      1
H-MC       X40        60949.0      2
L-LC       XY25       42085.0      1
H-MC       X40N       25821.0      1
M-LC       XY25       14702.0      1
H-SC       X40        13247.0      1
H-LC       XY25        6847.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
